In [1]:
import torch
import torch.nn as nn
import math
import numpy as np
import datetime

np.random.seed(42)

import sys
print(sys.executable) # just to check which python

import gym
from gym import spaces

from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

/usr/local/opt/python@3.9/bin/python3.9


In [2]:
class RequestType:
    def __init__(self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None):
        # distribution is 1x2 if elastic and 1x1 if static
        
        self.type = request_type
        self.bw = bandwidth
        self.service_rate = service_rate
        self.arrival_rate = arrival_rate
        self.source = source
        self.sink = sink
        self.distribution = distribution
        self.switch_rate = switch_rate
        
        self.num_made = 0
        self.num_accepted = 0
        
        if (request_type == "elastic"):
            self.scale_num_made = 0
            self.scale_num_accepted = 0

class Request:
    def __init__(self, request_type, service_time, arrival_time, source, sink, transfer_rate, distribution=None, parent_elastic=None, bw_dist=None, request_type_template=None):
        self.type = request_type
        self.service_time = service_time
        self.arrival_time = arrival_time
        self.source = source
        self.sink = sink
        self.bw = transfer_rate
        self.request_type = request_type
        self.parent_elastic = parent_elastic
        self.accepted = None
        self.path = None
        self.bw_dist = bw_dist
        self.expired = False
        
        self.blueprint = request_type_template
        
        if request_type == "elastic":
            self.distribution = distribution
            self.scale_requests = []
                   
        if request_type_template is not None:
            if request_type == "scale":
                request_type_template.scale_num_made += 1
            else:
                request_type_template.num_made += 1
    
    def process_acceptance(self):
        self.accepted = True
        if self.type == "scale":
            self.blueprint.scale_num_accepted += 1
        else:
            self.blueprint.num_accepted += 1
            
        if self.type == "elastic":
            self.blueprint.scale_num_made += len(self.scale_requests)
    
    def add_scale_request(self, req): 
        # we store related scale requests for elastic requests
        # not used if static request
        self.scale_requests.append(req)
            
    def get_encoding(self, nodes_in_environment):
        # as per our notes, this SHOULD return 1x5 tensor,
        # but we have one hot encodings INSIDE this tensor,
        # so we will flatten this and return, so the size will be
        # larger than 1x5
        
        # nodes_in_environment is a list of all the nodes in our graph
        # eg ["a", "b", "c"]
        
        # request is [one hot source, one hot destination, bw, service time, one hot type]
                
        one_hot_source = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.source)]), num_classes=len(nodes_in_environment)).flatten().float()
        one_hot_dest   = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.sink)]), num_classes=len(nodes_in_environment)).flatten().float()
    
        if self.request_type == "static":
            one_hot_type = torch.tensor([1, 0, 0])
        elif self.request_type == "elastic":
            one_hot_type = torch.tensor([0, 1, 0])
        elif self.request_type == "scale":
            one_hot_type = torch.tensor([0, 0, 1])
            
        encoding = torch.cat([one_hot_source, 
                             one_hot_dest,
                             torch.tensor([self.bw]).float(), 
                             torch.tensor([self.service_time]).float(),
                             one_hot_type.float()])
        
        return encoding

In [3]:
class Link:
    def __init__(self, node_1, node_2, bw_capacity):
        self.serving_requests = []
        self.nodes = [node_1, node_2]
        self.total_bw = bw_capacity
        
    def reset(self):
        self.serving_requests = []
        
    def add_request(self, request_obj):
        self.serving_requests.append(request_obj)
        
    def remove_request(self, request_obj):
        self.serving_requests.remove(request_obj)
        
    def remaining_bw(self): 
        # subtracting bw being used from total bw capacity
        bw_being_used = 0
        for req in self.serving_requests:
            bw_being_used += req.bw
            
        return (self.total_bw - bw_being_used)

In [4]:
class Environment(gym.Env):
    # requests_in_service_encoder = nn.RNN(????, 7)
    metadata = {'render.modes': ['human']}
    
    def __init__(self, nodes, links, request_blueprints, use_RNN=False, sb3_compat=False):
        super(Environment, self).__init__()
                
        """
        nodes: list of strings where each string is just a name or identifier of a node
        links: list of tuples where in tuple t, t[0] is first node, t[1] is another node, and t[2] is bw capacity of the link
        request_blueprints: list of DeploymentRequest objects
        """
        self.nodes = nodes
        self.links = {}
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.request_blueprints = request_blueprints
        self.last_time = 0
        self.episode_timesteps = 600
        self.use_RNN = use_RNN
        self.sb3_compat = sb3_compat
        self.precomputed_paths = {}
        
        if sb3_compat:
            self.request_being_considered = None
        
        for link in links:
            if link[0] not in self.nodes or link[1] not in self.nodes:
                raise Exception("Node in link " + str(link) + " doesn't exist")
            
            link_obj = Link(*link)

            self.links[link[0] + link[1]] = link_obj
            self.links[link[1] + link[0]] = link_obj
            
        self.precompute_paths()
            
        self.request_list = self.create_requests()
        self.request_queue = iter(self.request_list)
        
        # Setup gym-specific code
        env_encoding_size = self.get_encoding(increment_iterator=False).size()
        req_encoding_size = self.request_list[0].get_encoding(self.nodes).size()
        # print(env_encoding_size[0] + req_encoding_size[0])
        
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(low=-100, high=100,
                                      shape=(env_encoding_size[0] + req_encoding_size[0],), dtype=np.float32)
        
        # TODO, WRITE RNN logic
        
        #if use_RNN:
        #    self.requests_in_service_encoder = nn.RNN
        
    def precompute_paths(self):
        for req_type in self.request_blueprints:
            self.precomputed_paths[req_type.source + req_type.sink] = self.search(req_type.source, req_type.sink, [], [])
            
    def add_request(self, request, path=None): # we want to add this request to a link or path
        # path: a list of nodes that the request traverses including source and sink
        # if no path is specified, path is assumed to be [req.source, req.sink]
        
        if path is not None: 
            nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
            for node_pair in nodes:
                self.links[node_pair[0] + node_pair[1]].add_request(request)
        
        else:
            self.links[request.source + request.sink].add_request(request)
       
        self.request_history.append(request)
        # print(self.links[request.source + request.sink])
    
    def reset(self):
        for link in self.links.values():
            link.reset()
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.last_time = 0
        self.request_list = self.create_requests()
        self.request_queue = iter(self.request_list)
        self.precomputed_paths = {}

        self.precompute_paths()
        
        return self.get_encoding()
        
    def reward(self, request, decision):
        base_rate = 1         # 1 when static
        type_bonus = 0.9      # 0.9 when static
        bw = request.bw
        if request.type == "elastic":
            #base_rate = request.bw
            type_bonus = 1.1                # 1.1 when elastic
            bw = np.array(request.bw_dist).dot(request.distribution)
            
        r = bw * base_rate * request.service_time * type_bonus
        
        # if remaining bandwidth on link(s) < 0, very "bad" reward
        if request.path is not None:
            path_length = len(request.path)
            
            r *= math.pow(0.9, path_length - 2)
            
            nodes = [[request.path[i], request.path[i + 1]] for i in range(len(request.path) - 1)]
            for node_pair in nodes:
                if self.links[node_pair[0] + node_pair[1]].remaining_bw() < 0:
                    print("exceeded: " + str(-r * 10))
                    print(request.type)
                    return (-r * 10)
        else:
            # path is direct, so no decrease of reward needed
            remaining_bw = self.links[request.source + request.sink].remaining_bw()
            if remaining_bw < 0:
                print("exceeded: " + str(-r * 10))
                print(request.type)
                return (-r * 10)
        
        if decision == "accept":
            return r
        
        if decision == "reject":
            if request.type == "static" or request.type == "elastic":
                return 0
            elif request.type == "scale":
                if len(self.past_distributions) == 0:
                    return -1 * r
                
                else:
                    current_sum = torch.from_numpy(np.sum(self.past_distributions, axis=0))

                    average_past_distribution = current_sum / len(self.past_distributions)
                    current_req_distribution = torch.tensor(request.parent_elastic.distribution)

                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))

                """
                past_distributions = []
                for req in self.request_history:
                    if req.request_type == "elastic":
                        past_distributions.append(req.distribution)
                
                average_past_distribution = torch.mean(past_distributions, dim=1)
                current_req_distribution = torch.tensor(request.distribution)
                
                if bool(average_past_distribution[0] < current_req_distribution[0]):
                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))
                else:
                    return 0
                """
                
    def next_req(self):
        next_req = next(self.request_queue)
        if self.sb3_compat:
            self.request_being_considered = next_req
                    
        return next_req
                
    def step(self, action, req=None):
        # what happens if we have two requests that come in on the same timestep but there is only enough bandwidth for one?
        # do we the decision on the second request with knowledge of the first request
        # essentially, after we accept the first request, will we submit an updated encoding of the network to the policy network?
 
        # actions is a Nx2 matrix where the first column in the request and second is the decision
        # decision is either "accept" or "reject"
        # this is given by our agent
        
        # if req is None, that means we are using sb3_compat=True and we can get the req from self.request_being_considered
                            
        if req is None:
            req = self.request_being_considered
            
        if action == 1:
            # accept request
            if req.type == "scale":
                path = req.parent_elastic.path
            else:
                possible, path = self.get_path(req)
            
            self.add_request(req, path)
            req.process_acceptance()
            reward = self.reward(req, "accept")
        elif action == 0:
            # reject
            req.accepted = False
            reward = self.reward(req, "reject")

        obs = self.get_encoding()
        
        done = req.arrival_time > 600
        info = {}
        
        return obs, reward, done, info
        
    def update_requests(self, current_time):
        # here, we remove expired requests and update E_history based off of the request stats
        
        for link in self.links.values():
            for request in link.serving_requests.copy():
                if (request.arrival_time + request.service_time) < current_time:
                    # request has expired, let's remove it from the links
                    link.remove_request(request)

                    if request.type == "elastic" and not request.expired:
                        time_on_higher_bw = 0
                        for scale_req in request.scale_requests:
                            time_on_higher_bw += scale_req.service_time

                        time_on_lower_bw = request.service_time - time_on_higher_bw

                        # calculate E[history]
                        request_time = np.array([time_on_lower_bw, time_on_higher_bw])
                        request_bw = request.bw
                        result = (request_time / request_time.sum()).dot(request_bw)
                        self.past_distributions.append(request_time / request_time.sum())
                        self.E_history.append(result)
                        request.expired = True

    def get_encoding(self, increment_iterator=True):
        links_processed = [] 
        # these will store links that we have already encoded so we don't encode them again
        
        current_encoding = []
        
        # h = torch.zeros(7) # assuming 7 for h0 size
        # last_out = None
        
        env_encoding = []
        
        if increment_iterator:
            next_req = self.next_req()
            
            possible, path = self.get_path(next_req)

            while True:
                if next_req.type == "scale" and next_req.parent_elastic.accepted:
                    break
                elif next_req.type == "scale" and not next_req.parent_elastic.accepted:
                    next_req = self.next_req()
                    possible, path = self.get_path(next_req)
                    continue
                elif not possible:
                    next_req = self.next_req()
                    possible, path = self.get_path(next_req)
                    continue
                elif possible:
                    break
            
            # we will only use requests that are possible to accept OR are static w accepted deployment req

            """
            while next_req.type == "scale":
                if next_req.parent_elastic.accepted:
                    next_req.accepted = True # we must accept since we accepted elastic req
                    self.add_request(next_req, next_req.parent_elastic.path)
                next_req = self.next_req()
            """
            self.update_requests(next_req.arrival_time)

        for link in self.links.values():
            if link in links_processed:
                continue

                        
            # Commented because we don't want to encode any queue for phase 1
            
            # for req in link.serving_requests
                # request is [one hot source, one hot destination, bw, service time, one hot type]
                
                # one_hot_source = nn.functional.one_hot(torch.tensor([self.nodes.index(req.source)]), num_classes=len(self.nodes))
                # one_hot_dest   = nn.functional.one_hot(torch.tensor([self.nodes.index(req.sink)]), num_classes=len(self.nodes))

                # req_tensor = torch.Tensor([]) # mismatched dimensions??!
                # last_out, h = self.requests_in_service_encoder(req_tensor, h)

            # current_encoding.append(torch.cat(torch.Tensor([link.remaining_bw]), last_out))
            # torch.stack(current_encoding)
            
            # check implementation later
            
            env_encoding.append(link.remaining_bw())
            
            links_processed.append(link)
            
        if not increment_iterator:
            return torch.tensor(env_encoding)
        
        if self.sb3_compat:
            return torch.cat([torch.tensor(env_encoding).float(), torch.tensor(next_req.get_encoding(self.nodes)).float()])
        else:
            return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes)), next_req
    
    def create_requests(self):
        requests = []
        
        for request_type in self.request_blueprints:
            arrival_times = []
            service_times = []
            last_arrival = 0
        
            while last_arrival < self.episode_timesteps: # we want to generate requests till we reach episode end
                last_arrival += np.random.exponential(request_type.arrival_rate)
                arrival_times.append(last_arrival)
                                
            for _ in arrival_times:
                service_times.append(np.random.exponential(request_type.service_rate))
                
            for arrival_time, service_time in zip(arrival_times, service_times):
                # start creating requests
                
                new_request = Request(request_type.type, service_time, arrival_time, request_type.source, request_type.sink, 
                                      request_type.bw[0], request_type.distribution, bw_dist=request_type.bw, request_type_template=request_type)
                requests.append(new_request)
                
                if request_type.type == "elastic": 
                    # we will start with the first bandwidth element as starting bw
                    # WE ASSUME that bw[0] < bw[1]
                    timesteps_from_deployment = 0
                    current_bw = request_type.bw[0]
                    while timesteps_from_deployment < service_time:
                        if current_bw == request_type.bw[0]:
                            # we want to generate a scale request to increase bw
                            scale_bw = request_type.bw[1] - current_bw
                            scale_service_time = np.random.exponential(request_type.switch_rate[1])
                            scale_request = Request("scale", scale_service_time, \
                                                    arrival_time + timesteps_from_deployment, request_type.source, \
                                                   request_type.sink, scale_bw, parent_elastic=new_request,
                                                   request_type_template=request_type)
                            requests.append(scale_request)
                            new_request.add_scale_request(scale_request)
                            
                            timesteps_from_deployment += scale_service_time
                            current_bw = request_type.bw[1] # request_type.bw[0] + scale_bw
                        elif current_bw == request_type.bw[1]:
                            # we want to go to lower bw and spend some time there
                            time_spent_on_lower_bw = np.random.exponential(request_type.switch_rate[0])
                            timesteps_from_deployment += time_spent_on_lower_bw
                            current_bw = request_type.bw[0]
                            
        # sort requests by arrival time
        requests.sort(key=lambda x: x.arrival_time)
        return requests
    
    def search(self, source, dest, visited_a, paths):
        visited_a.append(source)
        # print(visited_a)

        for link in set(self.links.values()):
            visited = visited_a.copy()
            if source in link.nodes:
                if dest in link.nodes:
                    visited.append(dest)
                    paths.append(visited)

                x = link.nodes.copy()
                x.remove(source)
                if x[0] not in visited:
                    self.search(x[0], dest, visited.copy(), paths)
        return paths
    
    def print_statistics(self):
        for req_type in self.request_blueprints:
            print(req_type.source + " | " +
                   req_type.sink + " | " + 
                 "BW: " + str(req_type.bw) + " | " +
                 "Arrival rate: " + str(req_type.arrival_rate) + " | " +
                 "Acceptance rate: " + str(req_type.num_accepted / req_type.num_made))
            
            if req_type.type == "elastic":
                print("Scale accept rate for ^^: " + str(req_type.scale_num_accepted / req_type.scale_num_made))
            
        resources_used = 0
        for req in self.request_list:
            if req.accepted == True:
                resources_used += req.bw * req.service_time
            
        print("BW used: " + str(resources_used))
        
    def get_path(self, next_req_obj):
        # find all paths between source and sink
        paths = env.precomputed_paths[next_req_obj.source + next_req_obj.sink] 
        # (env.search(next_req_obj.source, next_req_obj.sink, [], []))
        paths.sort(key=lambda x: len(x)) # sort by shortest path
        selection = 0
        for path in paths:
            # check if this path works
            works = True
            nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
            for node_pair in nodes:
                if env.links[node_pair[0] + node_pair[1]].remaining_bw() < next_req_obj.bw:
                    works = False

            if works:
                selection = paths.index(path)
                selection_one_hot = nn.functional.one_hot(torch.tensor([selection]), num_classes=3).flatten()
                next_req_obj.path = path
                return True, path

        return False, None

In [5]:
def make_standard_env(sb3_compat=False):
    env = Environment(["a", "b", "c", "d", "e", "f"], [["a", "b", 10], ["a", "c", 10], ["b", "d", 10], \
                                                    ["c", "d", 20], ["c", "e", 10], ["d", "f", 10], \
                                                    ["e", "f", 10]], \
                          [RequestType("static", [2], 0.5, 0.75, "a", "b", [1]), \
                          RequestType("static", [8], 1, 1.5, "a", "b", [1]), \
                          RequestType("elastic", [4, 9], 1, 1.5, "a", "b", [0.8, 0.2], switch_rate=[0.08, 0.02]), \
                          RequestType("static", [1], 1, 1.5, "c", "d", [1]), \
                          RequestType("static", [7], 0.5, 0.75, "c", "d", [1]), \
                          RequestType("elastic", [3, 13], 2, 3, "c", "d", [0.9, 0.1], switch_rate=[0.09, 0.01]), \
                          RequestType("static", [3], 0.5, 0.75, "e", "f", [1]), \
                           RequestType("static", [6], 1, 1.5, "e", "f", [1]), \
                            RequestType("elastic", [5, 8], 2, 3, "e", "f", [0.7, 0.3], switch_rate=[0.07, 0.03])],
                         sb3_compat=sb3_compat)

    return env

### Choose shortest viable path

In [6]:
env = make_standard_env(sb3_compat=False)

In [7]:
def policy(env_encoding, next_req_encoding, next_req_obj):    
    return 1

In [9]:
# ONLY WORKS WITH sb3_compat=False when creating env object

total_reward = 0
env_encoding, next_req_encoding, next_req_obj = env.reset()
done = False

while not done:
    decision = policy(env_encoding, next_req_encoding, next_req_obj)
    
    obs, reward, done, info = env.step(decision, next_req_obj)
    env_encoding, next_req_encoding, next_req_obj = obs
    
    total_reward += reward

print(total_reward)

<ipython-input-4-ae3ac61764ff>:294: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes)), next_req


exceeded: -1.8056995950504973
scale
exceeded: -0.4986869710836567
scale
exceeded: -0.12605889797129557
scale
exceeded: -0.3525605105533618
scale
exceeded: -0.773993769237848
scale
exceeded: -1.741076158673989
scale
exceeded: -0.24241109737664426
scale
exceeded: -1.7460953715168617
scale
exceeded: -2.1496097030544576
scale
exceeded: -0.8528985337196118
scale
exceeded: -2.126395340971478
scale
exceeded: -0.1555670398960437
scale
exceeded: -0.2619393361219281
scale
exceeded: -0.4174520954673364
scale
exceeded: -1.1352675465727886
scale
exceeded: -2.1165009432688313
scale
exceeded: -0.06978132101891887
scale
exceeded: -2.7277100403921044
scale
exceeded: -3.763426907205928
scale
exceeded: -0.3256048189744701
scale
exceeded: -0.9597429684932155
scale
exceeded: -0.1867198769363791
scale
exceeded: -0.5759679478158543
scale
exceeded: -1.0108667113827607
scale
exceeded: -0.7118575220177347
scale
exceeded: -0.0018182003726414735
scale
exceeded: -1.9059026766590996
scale
exceeded: -0.0575999204718

exceeded: -0.8683151987882765
scale
exceeded: -0.5089333330024979
scale
exceeded: -0.5408927743612834
scale
exceeded: -0.2591065696878281
scale
exceeded: -0.3274257071815885
scale
exceeded: -0.7192198049060636
scale
exceeded: -0.10554108526095235
scale
exceeded: -0.9987063066262857
scale
exceeded: -3.98207960048712
scale
exceeded: -1.7793201945015813
scale
exceeded: -1.6387361663389564
scale
exceeded: -0.001866219805946015
scale
exceeded: -0.8435743174543291
scale
exceeded: -2.0109225387182863
scale
exceeded: -2.6746937979458223
scale
exceeded: -1.6379477966694778
scale
exceeded: -0.5909479861541058
scale
exceeded: -2.815241822993371
scale
exceeded: -0.04757808285973517
scale
exceeded: -0.7385157074578528
scale
exceeded: -1.7500823214603853
scale
exceeded: -1.1125391135919869
scale
exceeded: -0.8085969089687718
scale
exceeded: -1.348145893603667
scale
exceeded: -0.16154206054902726
scale
exceeded: -3.4280788553558033
scale
exceeded: -1.4778880981555615
scale
exceeded: -0.80647687958116

exceeded: -0.05239231782288774
scale
exceeded: -2.4734470861234312
scale
exceeded: -0.07050360424493893
scale
exceeded: -0.5575754538192913
scale
exceeded: -0.024379990651358194
scale
exceeded: -1.2788359679415162
scale
exceeded: -0.6918566377198675
scale
exceeded: -0.5467503992558749
scale
exceeded: -2.452234998583246
scale
exceeded: -0.8104953058025633
scale
exceeded: -1.3914302722406067
scale
exceeded: -0.45744711661720733
scale
exceeded: -0.5631866939142858
scale
exceeded: -0.2737004935835847
scale
exceeded: -0.012307823591803085
scale
exceeded: -3.556679937323815
scale
exceeded: -0.21514917756386376
scale
exceeded: -0.6471428454922229
scale
exceeded: -3.709101409269034
scale
exceeded: -1.6931892956917507
scale
exceeded: -1.4945414104979808
scale
exceeded: -0.6475926450536308
scale
exceeded: -0.35824437593138037
scale
exceeded: -1.4657049905094324
scale
exceeded: -0.4567725903166535
scale
exceeded: -0.19113452546332077
scale
exceeded: -0.4094053492838874
scale
exceeded: -0.39520259

exceeded: -0.10789751858411795
scale
exceeded: -0.3824875763649034
scale
exceeded: -0.9447703472367475
scale
exceeded: -1.5910256448018227
scale
exceeded: -0.3350786051803122
scale
exceeded: -0.6652515430479728
scale
exceeded: -1.5736929059619176
scale
exceeded: -0.3763420210602626
scale
exceeded: -0.19177417299839633
scale
exceeded: -2.369657193583517
scale
exceeded: -1.0997455343538576
scale
exceeded: -1.349442210280519
scale
exceeded: -1.2982521368071973
scale
exceeded: -0.22397711294464512
scale
exceeded: -0.03628504603846512
scale
exceeded: -2.2189686900639978
scale
exceeded: -0.2156842766199027
scale
exceeded: -0.04743582763145391
scale
exceeded: -2.6634256090449355
scale
exceeded: -0.006700406872974161
scale
exceeded: -1.1682464784103082
scale
exceeded: -1.4953397372535364
scale
exceeded: -1.2978128071215438
scale
exceeded: -1.2174787045283428
scale
exceeded: -0.3674673159650452
scale
exceeded: -1.0660783031503436
scale
exceeded: -0.37307347310928823
scale
exceeded: -1.714124993

...

exceeded: -2.4882309005476912
scale
exceeded: -0.8288986503885004
scale
exceeded: -0.10881078557165065
scale
exceeded: -0.4570813471974764
scale
exceeded: -1.1685931429591316
scale
exceeded: -0.5429082668411935
scale
exceeded: -0.9096474879597872
scale
exceeded: -0.014649396124281447
scale
exceeded: -0.7478029420284857
scale
exceeded: -0.4937241187374428
scale
exceeded: -0.24502221846682254
scale
exceeded: -0.4122005943040788
scale
exceeded: -2.2069245255483865
scale
exceeded: -0.098216847127353
scale
exceeded: -2.629336185151445
scale
exceeded: -1.684754127347881
scale
exceeded: -0.5174276426890211
scale
exceeded: -2.986332259959447
scale
exceeded: -0.2793342509988815
scale
exceeded: -0.16636309231123272
scale
exceeded: -0.7415825179422371
scale
exceeded: -1.4279509430778627
scale
exceeded: -0.029637343833370036
scale
exceeded: -0.23535021102712067
scale
exceeded: -0.19447318019763832
scale
exceeded: -0.24997613069881408
scale
exceeded: -1.1821452642980108
scale
exceeded: -0.176952421

exceeded: -1.858401313023409
scale
exceeded: -0.8938602295734657
scale
exceeded: -0.13381078421896683
scale
exceeded: -2.2673857133573154
scale
exceeded: -0.30460782309298823
scale
exceeded: -0.20439473061975025
scale
exceeded: -0.09143879563515254
scale
exceeded: -1.2906259340852178
scale
exceeded: -1.8301949526081427
scale
exceeded: -0.11145876681184988
scale
exceeded: -1.2104976690281875
scale
exceeded: -0.09850217578082027
scale
exceeded: -0.8520201925080482
scale
exceeded: -1.5139371862326363
scale
exceeded: -4.55458744247945
scale
exceeded: -0.9343776245923631
scale
exceeded: -0.3870537902107521
scale
exceeded: -0.3819975984579491
scale
exceeded: -0.013949415993506055
scale
exceeded: -0.3451480855692681
scale
exceeded: -0.5109599915511494
scale
exceeded: -0.04807932010821052
scale
exceeded: -1.2035589939582152
scale
exceeded: -0.6654946236606429
scale
exceeded: -0.1851753950743079
scale
exceeded: -0.3309781568761587
scale
exceeded: -0.3058179620163937
scale
exceeded: -1.833472401

exceeded: -1.3441027557534093
scale
exceeded: -0.25557634679675606
scale
exceeded: -0.9385652169754548
scale
exceeded: -0.03803152086075391
scale
exceeded: -1.7728423520839929
scale
exceeded: -1.559855821818773
scale
exceeded: -2.5094274693000616
scale
exceeded: -1.3497058684866217
scale
exceeded: -1.2275473150488605
scale
exceeded: -1.1605231706595278
scale
exceeded: -1.1335198276973393
scale
exceeded: -0.15971544746790028
scale
exceeded: -0.7178700647766221
scale
exceeded: -0.017866182985853324
scale
exceeded: -1.7123823952848882
scale
exceeded: -1.5479014273647502
scale
exceeded: -0.1478854908768301
scale
exceeded: -0.8218981676293444
scale
exceeded: -1.0282105508466168
scale
exceeded: -4.813619840489242
scale
exceeded: -0.3883489953651112
scale
exceeded: -1.9423694141961636
scale
exceeded: -0.5462455787773464
scale
exceeded: -0.3041898269084121
scale
exceeded: -1.3641854618281122
scale
exceeded: -0.22244682741878535
scale
exceeded: -0.46183521476349143
scale
exceeded: -1.3472233737

exceeded: -0.017239012080560164
scale
exceeded: -1.7372614921556615
scale
exceeded: -0.21425551751640537
scale
exceeded: -1.575797751469549
scale
exceeded: -3.415211195572999
scale
exceeded: -0.36281445348335495
scale
exceeded: -0.3753517287920049
scale
exceeded: -0.03984891108372193
scale
exceeded: -0.044977274628910234
scale
exceeded: -2.0964457416891586
scale
exceeded: -0.24712120149201355
scale
exceeded: -3.0053128971702288
scale
exceeded: -0.9612462300058504
scale
exceeded: -0.6845289413228904
scale
exceeded: -0.07398741728169717
scale
exceeded: -2.345464392611999
scale
exceeded: -2.306777045846325
scale
exceeded: -2.9604896982335664
scale
exceeded: -0.4136294262480947
scale
exceeded: -0.05718327882445583
scale
exceeded: -2.925596316561313
scale
exceeded: -0.2554397858931699
scale
exceeded: -1.1067088326663481
scale
exceeded: -3.700145544191651
scale
exceeded: -0.19118370748616362
scale
exceeded: -0.7939322094314629
scale
exceeded: -3.799649902064683
scale
exceeded: -1.58629627818

exceeded: -0.2408728061227117
scale
exceeded: -1.5327337591492107
scale
exceeded: -0.15532690536650198
scale
exceeded: -1.0215777503413848
scale
exceeded: -0.46871112717006697
scale
exceeded: -0.40284276911542233
scale
exceeded: -0.30242126921814266
scale
exceeded: -0.3940036351701893
scale
exceeded: -0.5256875304373059
scale
exceeded: -0.052830196100881105
scale
exceeded: -0.7489750570468302
scale
exceeded: -2.7784297586277926
scale
exceeded: -1.0276498569635653
scale
exceeded: -2.0949200816605127
scale
exceeded: -0.831937326815857
scale
exceeded: -0.16594380267871037
scale
exceeded: -0.6513753615925398
scale
exceeded: -0.37634686379776877
scale
exceeded: -0.83398717657612
scale
exceeded: -1.1044587518996756
scale
exceeded: -0.7923093539093948
scale
exceeded: -0.07767264503146579
scale
exceeded: -0.5355020649748374
scale
exceeded: -0.2539447857272317
scale
exceeded: -0.24139091002346308
scale
exceeded: -1.8659812927200614
scale
exceeded: -0.0026593406028615034
scale
exceeded: -1.91839

In [10]:
env.print_statistics()

a | b | BW: [2] | Arrival rate: 0.75 | Acceptance rate: 0.5020374898125509
a | b | BW: [8] | Arrival rate: 1.5 | Acceptance rate: 0.29520605550883094
a | b | BW: [4, 9] | Arrival rate: 1.5 | Acceptance rate: 0.4124293785310734
Scale accept rate for ^^: 0.29478843653095926
c | d | BW: [1] | Arrival rate: 1.5 | Acceptance rate: 0.5433673469387755
c | d | BW: [7] | Arrival rate: 0.75 | Acceptance rate: 0.36799017601309864
c | d | BW: [3, 13] | Arrival rate: 3 | Acceptance rate: 0.4974182444061962
Scale accept rate for ^^: 0.32933038184836744
e | f | BW: [3] | Arrival rate: 0.75 | Acceptance rate: 0.4983471074380165
e | f | BW: [6] | Arrival rate: 1.5 | Acceptance rate: 0.33250414593698174
e | f | BW: [5, 8] | Arrival rate: 3 | Acceptance rate: 0.36180124223602483
Scale accept rate for ^^: 0.25191415938746897
BW used: 12150.22440526122


### PPO

In [10]:
def make_env(rank, seed=42):        
    def _init():
        env = make_standard_env(sb3_compat=True)
        env.seed(seed + rank)
        return env

    set_random_seed(seed)
    return _init

env = SubprocVecEnv([make_env(i) for i in range(4)])
env
                # self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None
    

In [23]:
env = make_standard_env(sb3_compat=True)

In [24]:
policy_kwargs = dict(activation_fn=nn.ReLU,
                     net_arch=[dict(pi=[64, 64, 64], vf=[64, 64, 64])])
model = PPO(MlpPolicy, env, verbose=1, policy_kwargs=policy_kwargs, tensorboard_log="./multilink/", device="cpu")
model.learn(total_timesteps=100000000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./multilink/PPO_25
exceeded: -0.919942855642051
scale


<ipython-input-4-ae3ac61764ff>:292: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat([torch.tensor(env_encoding).float(), torch.tensor(next_req.get_encoding(self.nodes)).float()])
/usr/local/lib/python3.9/site-packages/torch/nn/functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


exceeded: -0.3682313430293596
scale
exceeded: -0.429701407124058
scale
exceeded: -3.1806640464136753
scale
exceeded: -0.6748100768312136
scale
exceeded: -0.13874907882063126
scale


exceeded: -0.454768447807572
scale
exceeded: -0.1421416882330521
scale
exceeded: -1.2535376981492472
scale
exceeded: -0.8911021911845464
scale
exceeded: -0.2713765966769156
scale
exceeded: -1.5258529985808917
scale
exceeded: -0.6617691702197015
scale
exceeded: -0.9107588693263797
scale
exceeded: -0.08285975250065068
scale
exceeded: -0.1518191967346082
scale
exceeded: -0.3626843423335039
scale
exceeded: -1.2474149533261523
scale
exceeded: -0.07986557868160131
scale
exceeded: -0.5800795937788963
scale


exceeded: -0.9017481800344298
scale
exceeded: -2.1197612121826808
scale
exceeded: -1.0517184281298662
scale
exceeded: -1.8823764602446358
scale
exceeded: -1.9729837789871885
scale
exceeded: -0.6607325780783109
scale
exceeded: -0.022894719928457798
scale


...

exceeded: -0.12100079792696841
scale
exceeded: -1.367800196891785
scale


exceeded: -2.1412375477175307
scale
exceeded: -0.4942555132513646
scale
exceeded: -0.17985195461547976
scale
exceeded: -0.4367496639870341
scale
exceeded: -2.436537364949211
scale
exceeded: -0.5210559750930226
scale
exceeded: -2.5613243438988738
scale
exceeded: -1.6726505994416825
scale
exceeded: -0.14112259572374808
scale
exceeded: -2.916186525100499
scale
exceeded: -0.23302632101224005
scale
exceeded: -0.30181146734229025
scale
exceeded: -0.8643859810880783
scale
exceeded: -0.14209746015048405
scale
exceeded: -0.4485238661864516
scale
exceeded: -3.0770964760216533
scale
exceeded: -0.360144645569476
scale
exceeded: -0.09585155067079218
scale
exceeded: -0.6861529741319694
scale
exceeded: -0.3003814186887225
scale
exceeded: -0.919610139483553
scale
exceeded: -0.4723448034508787
scale
exceeded: -0.4802892945501763
scale
exceeded: -2.650386481935313
scale
exceeded: -0.7808646442423133
scale
exceeded: -2.051091297505256
scale
exceeded: -3.3016345912954193
scale
exceeded: -0.180772632775959

exceeded: -0.7098873812773082
scale
exceeded: -0.2714822341556743
scale
exceeded: -0.6229042474814598
scale
exceeded: -0.027243630318897268
scale
exceeded: -1.2770749830911239
scale
exceeded: -0.7631566134344268
scale
exceeded: -0.5304065721064903
scale
exceeded: -0.0003307855755048369
scale
exceeded: -0.04773760887875706
scale
exceeded: -0.5347018537839993
scale
exceeded: -2.328350823911694
scale
exceeded: -0.8706833430996196
scale
exceeded: -0.08098066782946903
scale
exceeded: -0.7532149629281479
scale
exceeded: -2.091347911359981
scale
exceeded: -0.7411420375115545
scale
exceeded: -2.5923592373671047
scale
exceeded: -0.9346629097688728
scale
exceeded: -0.10559883485221173
scale
exceeded: -2.067283433291726
scale
exceeded: -1.5468474550457034
scale
exceeded: -2.555385364114515
scale
exceeded: -0.5158457967226842
scale
exceeded: -0.34380152941496445
scale
exceeded: -0.20804752789828754
scale
exceeded: -0.9833173029333271
scale
exceeded: -1.1865916564159193
scale


exceeded: -1.2631418367334446
scale
exceeded: -0.8227684620176305
scale
exceeded: -0.9885495699097581
scale
exceeded: -0.19166124966135994
scale
exceeded: -0.4312776598514716
scale
exceeded: -0.19567790448028616
scale
exceeded: -2.3318902486461046
scale
exceeded: -0.28205383192780265
scale
exceeded: -0.43487665409601967
scale
exceeded: -0.09862420531889157
scale
exceeded: -0.6068887297686266
scale
exceeded: -0.06036758621459585
scale
exceeded: -0.003895323273666408
scale
exceeded: -3.11004692249387
scale
exceeded: -1.5758728769520045
scale
exceeded: -0.7023057944178515
scale
exceeded: -0.9269282459918925
scale
exceeded: -0.7426154132496512
scale
exceeded: -0.42216643472906407
scale
exceeded: -0.027259268817387
scale
exceeded: -2.546850279747458
scale
exceeded: -0.7830708134103258
scale
exceeded: -2.845204928693468
scale


KeyboardInterrupt: 

In [11]:
env = make_standard_env(sb3_compat=True)

In [14]:
total_reward = 0
observation = env.reset()
done = False
while not done:
    action, _states = model.predict(observation, deterministic=True)
        
    observation, reward, done, info = env.step(action)
        
    total_reward += reward
    
print(total_reward)

<ipython-input-4-6c92528c5ad0>:281: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat([torch.tensor(env_encoding).float(), torch.tensor(next_req.get_encoding(self.nodes)).float()])


exceeded: -0.13142528692712474
exceeded: -0.14775145690201816
exceeded: -0.10371693925766648
exceeded: -0.059099656926096006
exceeded: -0.8749381029437504
exceeded: -0.039091305817006025
exceeded: -0.26771513207064146
exceeded: -1.6402572134419158
exceeded: -2.1014860272366125
exceeded: -19.497560475305093
exceeded: -0.7763375636576899


KeyboardInterrupt: 

In [ ]:
env.print_statistics()

In [ ]:
model.learn(total_timesteps=6000000, reset_num_timesteps=False)

In [ ]:
model.save("3M_trained")

## TD3

In [11]:
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [12]:
env = make_standard_env(sb3_compat=True)

env

In [13]:
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

In [ ]:
model = TD3("MlpPolicy", env, action_noise=action_noise, verbose=1, tensorboard_log="./multilink/", device="cuda")
model.learn(total_timesteps=100000000)

In [ ]:
total_reward = 0
observation = env.reset()
done = False
while not done:
    action, _states = model.predict(observation, deterministic=True)

    observation, reward, done, info = env.step(action)
    print(observation)
    
    total_reward += reward
    
print(total_reward)

In [8]:
spaces.Discrete(2).sample()

0

In [22]:
sum = 0
for i in range(1, 100000):
    sum += np.random.exponential(0.02)
    
print(sum / 100000)

0.019931865490254993


In [28]:
spaces.Box(low=-1, high=1, shape=([1]), dtype=np.float32).sample()[0]

-0.5447028